---
# Algoritmos para Big Data – Projeto
## Parte 1: Leitura e Validação de Dados

**Dataset:** Flight Delay Dataset (Kaggle)

**Autores:**
- Henrique Niza (131898)
- Paulo Francisco Pinto (128962)
- Rute Roque (128919)

Spark Setup

In [2]:
# Importação das bibliotecas necessárias
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
# Criação da SparkSession
spark = SparkSession.builder.appName("FlightDelayValidation").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/05/24 19:49:19 WARN Utils: Your hostname, MacBook-Pro-de-admin.local, resolves to a loopback address: 127.0.0.1; using 192.168.68.50 instead (on interface en0)
25/05/24 19:49:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/24 19:49:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/24 19:49:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Leitura, Combinação e Amostragem dos Datasets (2018–2022)

In [5]:
# Lista de ficheiros CSV para cada ano
anos = ["2018", "2019", "2020", "2021", "2022"]
caminho_base = "../data/raw/Combined_Flights_"

# Leitura e união dos datasets anuais
dataframes = [spark.read.csv(f"{caminho_base}{ano}.csv", header=True, inferSchema=True) for ano in anos]
data = dataframes[0]
for df in dataframes[1:]:
    data = data.unionByName(df)

# Aplicar amostragem aleatória de 30% aos dados combinados
data = data.sample(fraction=0.3, seed=42)

In [ ]:
# Verificação do Schema e Amostras
data.printSchema()
data.show(5)
print(f"Número total de linhas (30% da amostra): {data.count()}")

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

25/05/24 19:54:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|          Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDelay|ArrTime|ArrD

Número total de linhas (30% da amostra): 8757283


In [ ]:
# Mostragem de número de colunas
colunas = data.columns
print(f"Número de colunas: {len(colunas)}")
print(colunas)

Número de colunas: 61
['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrDelay', 'ArrDel15'

In [ ]:
# Exemplo de mostragem: 5 linhas de dados
sample_rows = data.take(5)
sample_rows

[Row(FlightDate=datetime.date(2018, 1, 26), Airline='Endeavor Air Inc.', Origin='ABY', Dest='ATL', Cancelled=False, Diverted=False, CRSDepTime=1202, DepTime=1150.0, DepDelayMinutes=0.0, DepDelay=-12.0, ArrTime=1253.0, ArrDelayMinutes=0.0, AirTime=35.0, CRSElapsedTime=62.0, ActualElapsedTime=63.0, Distance=145.0, Year=2018, Quarter=1, Month=1, DayofMonth=26, DayOfWeek=5, Marketing_Airline_Network='DL', Operated_or_Branded_Code_Share_Partners='DL_CODESHARE', DOT_ID_Marketing_Airline=19790, IATA_Code_Marketing_Airline='DL', Flight_Number_Marketing_Airline=3298, Operating_Airline='9E', DOT_ID_Operating_Airline=20363, IATA_Code_Operating_Airline='9E', Tail_Number='N800AY', Flight_Number_Operating_Airline=3298, OriginAirportID=10146, OriginAirportSeqID=1014602, OriginCityMarketID=30146, OriginCityName='Albany, GA', OriginState='GA', OriginStateFips=13, OriginStateName='Georgia', OriginWac=34, DestAirportID=10397, DestAirportSeqID=1039707, DestCityMarketID=30397, DestCityName='Atlanta, GA', D

In [ ]:
# Remoção de Duplicados
print(f"Linhas antes: {data.count()} | Depois de dropDuplicates(): {data.dropDuplicates().count()}")
data = data.dropDuplicates()

Linhas antes: 8757283 | Depois de dropDuplicates(): 8757283


In [11]:
# Tratamento de Valores Nulos
null_counts = data.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in data.columns
]).collect()[0].asDict()
null_counts

{'FlightDate': 0,
 'Airline': 0,
 'Origin': 0,
 'Dest': 0,
 'Cancelled': 0,
 'Diverted': 0,
 'CRSDepTime': 0,
 'DepTime': 228993,
 'DepDelayMinutes': 229412,
 'DepDelay': 229412,
 'ArrTime': 236269,
 'ArrDelayMinutes': 254198,
 'AirTime': 256141,
 'CRSElapsedTime': 7,
 'ActualElapsedTime': 254025,
 'Distance': 0,
 'Year': 0,
 'Quarter': 0,
 'Month': 0,
 'DayofMonth': 0,
 'DayOfWeek': 0,
 'Marketing_Airline_Network': 0,
 'Operated_or_Branded_Code_Share_Partners': 0,
 'DOT_ID_Marketing_Airline': 0,
 'IATA_Code_Marketing_Airline': 0,
 'Flight_Number_Marketing_Airline': 0,
 'Operating_Airline': 0,
 'DOT_ID_Operating_Airline': 0,
 'IATA_Code_Operating_Airline': 0,
 'Tail_Number': 80290,
 'Flight_Number_Operating_Airline': 0,
 'OriginAirportID': 0,
 'OriginAirportSeqID': 0,
 'OriginCityMarketID': 0,
 'OriginCityName': 0,
 'OriginState': 0,
 'OriginStateFips': 0,
 'OriginStateName': 0,
 'OriginWac': 0,
 'DestAirportID': 0,
 'DestAirportSeqID': 0,
 'DestCityMarketID': 0,
 'DestCityName': 0,
 '

In [12]:
data = data.dropna()
print(f"Linhas após remoção de nulos: {data.count()}")

Linhas após remoção de nulos: 8500634


In [14]:
# Guardar Resultado Intermédio
output_path = "../data/processed/flights_cleaned_sample.parquet"
data.write.mode("overwrite").parquet(output_path)

25/05/24 20:15:42 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/05/24 20:15:46 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/05/24 20:15:50 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/05/24 20:15:54 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/05/24 20:15:58 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


Dados amostrados (30%) e limpos de 2018 a 2022 foram guardados como Parquet para uso nas próximas etapas do projeto.

In [15]:
print(f"Parquet salvo com sucesso em: {output_path}")

Parquet salvo com sucesso em: ../data/processed/flights_cleaned_sample.parquet
